In [7]:
import pandas as pd
import os
import numpy as np
import datetime as dt
import string
import csv

In [8]:
import gensim  
#from keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
#import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [10]:
# loading data

In [11]:
cwd = os.getcwd()

In [24]:
X_train= pd.read_csv(cwd+'/data/final/X_train.csv')
y_train= pd.read_csv(cwd+'/data/final/y_train.csv')

X_val= pd.read_csv(cwd+'/data/final/X_val.csv')
y_val= pd.read_csv(cwd+'/data/final/y_val.csv')

X_test= pd.read_csv(cwd+'/data/final/X_test.csv')
y_test= pd.read_csv(cwd+'/data/final/y_test.csv')

In [25]:
X_train['all_words']=X_train['title']+' '+ X_train['selftext']
X_test['all_words']=X_test['title']+' '+ X_test['selftext']
X_val['all_words']=X_val['title']+' '+ X_val['selftext']

In [26]:

y_train=X_train['engagement_score_std']
y_val=X_val['engagement_score_std']
y_test=X_test['engagement_score_std']

X_train=X_train[['all_words']]
X_val=X_val[['all_words']]
X_test=X_test[['all_words']]


In [27]:
#preprocessing:
# lowercases, tokenizes
X_train['all_words_tkn'] =X_train['all_words'].apply(simple_preprocess)
X_val['all_words_tkn'] =X_val['all_words'].apply(simple_preprocess)
X_test['all_words_tkn'] =X_test['all_words'].apply(simple_preprocess)

#remove stop words
stop_words = set(stopwords.words('english'))
add_stopwords={} #placeholder
stop_words.update(add_stopwords)
lmtzr = WordNetLemmatizer()

def stpout(lst):
    return [word for word in lst if word not in stop_words]

X_train['all_words_tkn']  =X_train['all_words_tkn'].apply(stpout)
X_train['all_words_lemm']=X_train['all_words_tkn'].apply(lambda x: [lmtzr.lemmatize(word) for word in x])

X_val['all_words_tkn']  =X_val['all_words_tkn'].apply(stpout)
X_val['all_words_lemm']=X_val['all_words_tkn'].apply(lambda x: [lmtzr.lemmatize(word) for word in x])

X_test['all_words_tkn']  =X_test['all_words_tkn'].apply(stpout)
X_test['all_words_lemm']=X_test['all_words_tkn'].apply(lambda x: [lmtzr.lemmatize(word) for word in x])


In [28]:
X_train['all_words_lemm']=X_train['all_words_lemm'].str.join(',').str.replace(',', ' ')
X_val['all_words_lemm']=X_val['all_words_lemm'].str.join(',').str.replace(',', ' ')
X_test['all_words_lemm']=X_test['all_words_lemm'].str.join(',').str.replace(',', ' ')

max_df float or int, default=1.0
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float in range [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

min_df float or int, default=1
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float in range of [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [29]:
# Extracting features from the training data using a sparse vectorizer

vectorizer = TfidfVectorizer(max_df=0.9, min_df=100)
X_train_vector = vectorizer.fit_transform(X_train['all_words_lemm'])


# Extracting features from the test data using the same vectorizer
X_val_vector = vectorizer.transform(X_val['all_words_lemm'])
X_test_vector = vectorizer.transform(X_test['all_words_lemm'])

feature_names = vectorizer.get_feature_names_out()


In [30]:

reg = LinearRegression().fit(X_train_vector, y_train)


In [31]:
y_train_pred= reg.predict(X_train_vector)
y_test_pred= reg.predict(X_test_vector)
y_val_pred= reg.predict(X_val_vector)

MSE_train = mean_squared_error(y_train,y_train_pred)
MSE_val = mean_squared_error(y_val,y_val_pred)
MSE_test = mean_squared_error(y_test,y_test_pred)

print('MSE for train set', MSE_train)
print('MSE for val set', MSE_val)
print('MSE for test set', MSE_test)



MSE for train set 0.7749349832951868
MSE for val set 0.7331650094495095
MSE for test set 0.3931705166238747
